<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Store Sales Forecasting with Prophet using Script Table Operator
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>
<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>

<p style = 'font-size:16px;font-family:Arial'>Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.<br>
<br>  
Our Main Objective is to predict sales of store in a week. We are using the python Prophet model and using the <b><u>S</u></b>cript <b><u>T</u></b>able <b><u>O</u></b>perator<b>(STO)</b> of Vantage for forecasting the Store Sales.
<br>
 <img src="./images/STO.png" width="800" align="right" style="padding: 40px; border: 4px solid; border-color:#404040; border-radius: 10px;"> 
    
    
<p style = 'font-size:16px;font-family:Arial'>STO allows you to leverage the parallelism of Vantage by installing external routines into each unit of parallelism. During the execution of the analysis, data is piped to and from those external routines in dozens, hundreds or thousands units of parallelism depending on the size of the production platform. This allows you the flexibility and power to extend the analytic power of Vantage with routines written in languages such as Python or R to process data in parallel without exporting to another platform.</p>
<p style = 'font-size:16px;font-family:Arial'>The SCRIPT function enables invoking the external language interpreters to execute your script files. It does so by running the interpreter in a shell environment invoked upon executing an STO query. Script execution takes place in protected mode, that is, outside the Advanced SQL Engine Database.
Pipelines the database I/O to R or Python scripts executed via Linux command line through STO</p>

<p style = 'font-size:16px;font-family:Arial'><b>Processing steps</b>
<li style = 'font-size:14px;font-family:Arial'>User “installs” their script file on Vantage </li>
    <li style = 'font-size:14px;font-family:Arial'>User executes a SQL query with STO</li>
<li style = 'font-size:14px;font-family:Arial'>STO invokes R or Python, passing the installed script</li>
<li style = 'font-size:14px;font-family:Arial'>Script executes in a Linux “forked” child process</li>
    <li style = 'font-size:14px;font-family:Arial'>Script reads from STDIN and writes to STDOUT</li>
    <li style = 'font-size:14px;font-family:Arial'>Script runs on each unit of parallelism</li>
<li style = 'font-size:14px;font-family:Arial'>Each unit of parallelism returns its own results independently</li>
</p>

<p></p>    
<br>  
<p style = 'font-size:16px;font-family:Arial'>Hence as a data science consultant, we are showcasing the complete approach about how we can make prediction of sales for different stores in advance. We are demonstrating how we can train our models and use them for scoring using the ClearScape Analytics platform. The data we are using is a sample dataset and the results and predictions may not be entirely accurate.
</p>



<p style = 'font-size:20px;font-family:Arial'><b>Data</b></p>
<p style = 'font-size:16px;font-family:Arial'>The dataset contains historical sales data for 1,115 Rossmann stores. The task is to forecast the "Sales" column for the test set. Note that some stores in the dataset were temporarily closed for refurbishment.</p>

<p style = 'font-size:16px;font-family:Arial'>Most of the fields are self-explanatory. The following are descriptions for those that aren't.</p>

<li style = 'font-size:16px;font-family:Arial'>Store - a unique Id for each store</li>
<li style = 'font-size:16px;font-family:Arial'>Sales - the turnover for any given day (this is what you are predicting)</li>
<li style = 'font-size:16px;font-family:Arial'>Customers - the number of customers on a given day</li>
<li style = 'font-size:16px;font-family:Arial'>Open - an indicator for whether the store was open: 0 = closed, 1 = open</li>
<li style = 'font-size:16px;font-family:Arial'>StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None</li>
<li style = 'font-size:16px;font-family:Arial'>SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools</li>
<li style = 'font-size:16px;font-family:Arial'>StoreType - differentiates between 4 different store models: a, b, c, d</li>
<li style = 'font-size:16px;font-family:Arial'>Assortment - describes an assortment level: a = basic, b = extra, c = extended</li>
<li style = 'font-size:16px;font-family:Arial'>CompetitionDistance - distance in meters to the nearest competitor store</li>
<li style = 'font-size:16px;font-family:Arial'>CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened</li>
<li style = 'font-size:16px;font-family:Arial'>Promo - indicates whether a store is running a promo on that day</li>
<li style = 'font-size:16px;font-family:Arial'>Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating</li>
<li style = 'font-size:16px;font-family:Arial'>Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2</li>
<li style = 'font-size:16px;font-family:Arial'>PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g., "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store.</li>
</p>

<hr style="height:2px;border:none">
<p style = 'font-size:20px;font-family:Arial'><b>1. Connect to Vantage.</b></p>

<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).</p>

In [ ]:
%%capture
!pip install statsmodels
# !pip install prophet

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial'><b>Note: </b><i>The above statements may need to be uncommented if you run the notebooks on a platform other than ClearScape Analytics Experience that does not have the libraries installed. If you uncomment those installs, be sure to restart the kernel after executing those lines to bring the installed libraries into memory. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b> and then clicking <b>Restart</b></i>.</p>
</div>

In [ ]:
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
import getpass
import pandas as pd

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from time import time
import os
from prophet import Prophet
import warnings
import itertools

import pickle
import base64

from teradataml import *

display.max_rows=5


<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, then use down arrow to go to next cell. Begin running steps with Shift + Enter keys.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=Store_Sales_Forecasting_Prophet_STO_PY_SQL.ipynb;' UPDATE FOR SESSION; ''')

<hr style="height:2px;border:none">
<p style = 'font-size:20px;font-family:Arial'><b>2. Getting Data for This Demo </b></p>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage.  You have the option of either running the demo using foreign tables to access the data without using any storage on your environment or downloading the data to local storage which may yield somewhat faster execution, but there could be considerations of available storage.  There are two statements in the following cell, and one is commented out.  You may switch which mode you choose by changing the comment string. 


In [ ]:
# %run -i ../run_procedure.py "call get_data('DEMO_ProphetSTO_cloud');"
 # Takes about 25 seconds
%run -i ../run_procedure.py "call get_data('DEMO_ProphetSTO_local');"
 # Takes about 70 seconds

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<hr style="height:2px;border:none">
<p style = 'font-size:20px;font-family:Arial'><b>3. Prepare data to do some basic Analysis of the Sales data.</b></p>

<p style = 'font-size:16px;font-family:Arial'>We create dataframe for the Stores and the Sales Data using tables from Vantage. To gain insights into the data's characteristics, we display a sample of 5 rows each.</p>

In [ ]:
store=DataFrame(in_schema('DEMO_ProphetSTO','Store'))
store  

<p style = 'font-size:16px;font-family:Arial'>The Store dataset contains description of the Stores like, StoreType, distance from the Competition Store and also various Promotion codes and Details.</p>

In [ ]:
sales=DataFrame(in_schema('DEMO_ProphetSTO','Sales_Data'))
sales  

<p style = 'font-size:16px;font-family:Arial'>The Store Sales dataset contains the Store, DayofWeek, Date of Sales , Sales done, Customer involved, SalesOpen is a flag mentioning if the Store is Open or Closed and Promotion Code applied for the Sales.</p>

<hr style="height:2px;border:none">
<p style = 'font-size:20px;font-family:Arial'><b>4. Data Analysis and Transformation </b></p>
<p style = 'font-size:18px;font-family:Arial'>In this first section we go through the Sales and store data, handle missing values and create new features for further analysis.</p>
<p style = 'font-size:16px;font-family:Arial'>We check the missing values for the CompetitionDistance column and replace it with the median values.</p>

In [ ]:
from teradataml import SimpleImputeFit, SimpleImputeTransform
fit_obj = SimpleImputeFit(data=store,
                              stats_columns="CompetitionDistance",
                              stats="median")

In [ ]:
obj =  SimpleImputeTransform(data=store,
                                 object=fit_obj.output)
 

In [ ]:
store=obj.result
store

<p style = 'font-size:16px;font-family:Arial'>We join the Store and Sales dataset to get the required columns for our analysis.</p>

In [ ]:
sales_store = sales.merge(right = store, how = "inner", on = "store=store",lsuffix='l', rsuffix='r')
sales_store=sales_store.assign(Store=sales_store.Store_l)
sales_store=sales_store.drop(['Store_l', 'Store_r'], axis=1)
sales_store

In [ ]:
sales_store.shape

<p style = 'font-size:16px;font-family:Arial'>The final dataset used for analysis contains 18 columns and 91,256 rows.</p>
<p style = 'font-size:16px;font-family:Arial'>Based on the data available we do some transformations on the data and create various features. From the SalesDate we, generate columns like , Year, Month, DayOfWeek , WeekofYear etc. Using the columns related to Competition like CompetionOpenSinceYear and CompetitionOpenSinceMonth we calculate if the Competition Store is Open or not(CompetitionOpen). Similarly, we do the processing for Promotions and create a flag(PromoOpen)</p>

In [ ]:
sales_store = sales_store.assign(CompetitionOpenSinceYear = 
                                     case([(sales_store.CompetitionOpenSinceYear.isnull() == True, '0')], else_ = sales_store.CompetitionOpenSinceYear),
                                CompetitionOpenSinceMonth = 
                                     case([(sales_store.CompetitionOpenSinceMonth.isnull() == True, '0')], else_ = sales_store.CompetitionOpenSinceMonth),
                                Promo2SinceYear = 
                                     case([(sales_store.Promo2SinceYear.isnull() == True, '0')], else_ = sales_store.Promo2SinceYear),
                                Promo2SinceWeek = 
                                     case([(sales_store.Promo2SinceWeek.isnull() == True, '0')], else_ = sales_store.Promo2SinceWeek)
                             )

In [ ]:
sales_store = sales_store.assign(Year = sales_store.SalesDate.year(),
                                 Month = sales_store.SalesDate.month(),
                                 Day = sales_store.SalesDate.day_of_month(),
                                 DayOfWeek = sales_store.SalesDate.day_of_week(),
                                 WeekOfYear = sales_store.SalesDate.week_of_year())

sales_store = sales_store.assign(CompetitionOpen = 12 * (sales_store.Year - sales_store.CompetitionOpenSinceYear)+
                                                     (sales_store.Month - sales_store.CompetitionOpenSinceMonth),
                                PromoOpen = 12 * (sales_store.Year - sales_store.Promo2SinceYear)+
                                                 (sales_store.WeekOfYear - sales_store.Promo2SinceWeek) / 4.0)


sales_store = sales_store.assign(CompetitionOpen = case([(sales_store.CompetitionOpen > 0, sales_store.CompetitionOpen)], else_ = 0),
                                PromoOpen = case([(sales_store.PromoOpen > 0, sales_store.PromoOpen)], else_ = 0))
    



In [ ]:
sales_store = sales_store.assign(StoreType = case([(sales_store.StoreType == '0', 0),(sales_store.StoreType == 'a', 1),
                                                  (sales_store.StoreType == 'b', 2),(sales_store.StoreType == 'c', 3),
                                                  (sales_store.StoreType == 'd', 4)]),
                                Assortment = case([(sales_store.Assortment == '0', 0),(sales_store.Assortment == 'a', 1),
                                                  (sales_store.Assortment == 'b', 2),(sales_store.Assortment == 'c', 3),
                                                  (sales_store.Assortment == 'd', 4)]),
                                StateHoliday = case([(sales_store.StateHoliday == '0', 0),(sales_store.StateHoliday == 'a', 1),
                                                  (sales_store.StateHoliday == 'b', 2),(sales_store.StateHoliday == 'c', 3),
                                                  (sales_store.StateHoliday == 'd', 4)])
                                
                                )  
                                

In [ ]:
sales_store = sales_store.assign(monthStr = case([(sales_store.Month == 1, 'Jan'),(sales_store.Month == 2, 'Feb'),
                                                  (sales_store.Month == 3, 'Mar'),(sales_store.Month == 4, 'Apr'),
                                                  (sales_store.Month == 5, 'May'),(sales_store.Month == 6, 'Jun'),
                                                  (sales_store.Month == 7, 'Jul'),(sales_store.Month == 8, 'Aug'),
                                                  (sales_store.Month == 9, 'Sep'),(sales_store.Month == 10, 'Oct'),
                                                  (sales_store.Month == 11,' Nov'),(sales_store.Month == 12, 'Dec')]),
                                IsPromoMonth = 0)

In [ ]:
sales_store

In [ ]:
plot_sales = sales_store.select(['Month','Sales']).groupby('Month').mean()
plot =  plot_sales.plot(x=plot_sales.Month, y=plot_sales.mean_Sales,
                           kind='bar', xlabel='Month', ylabel='Sales', color="orange")
 
# Display the plot.
plot.show()

<p style = 'font-size:16px;font-family:Arial'>The above graph shows the total sales across months for all stores. We can see that the sales are highest in December which is the Holiday Season.</p>
<p style = 'font-size:16px;font-family:Arial'>Now we will see the same metrics across different Store types and also based on whether there was any Promotion available(Promo=1) or not (Promo=0)

In [ ]:
#Catplot month Vs Sales
features_df = sales_store.to_pandas(all_rows=True)
sns.catplot(data = features_df, x = 'Month', y = "Sales", 
               col = 'StoreType', # per store type in cols
               palette = 'plasma',
               # hue = 'StoreType',
               row = 'Promo' # per promo in the store in rows
               # color ='Year'
           ) 

<p style = 'font-size:16px;font-family:Arial'>The above graph shows the Sales per Month for each of the 4 StoreTypes(a,b,c,d) for all the 1,115 Stores. The Top row shows the sales for Promo=0 and the bottom row is for Promo=1. Each dot represents the sum of sales for a particular store in a month depending on the Store Type and Promo Code. We can see that there are peaks mainly during the Year end period.</p>
<p style = 'font-size:16px;font-family:Arial'>All store types follow the same trend but at different scales depending on the presence of the promotion `Promo` and `StoreType` except for the StoreType = b.
</p>



<p style = 'font-size:16px;font-family:Arial'>Next we try to get four stores from store types to represent their group:</p>
<li style = 'font-size:16px;font-family:Arial'> Store number 2 for `StoreType` A</li>
<li style = 'font-size:16px;font-family:Arial'>Store number 85 for `StoreType` B</li>
<li style = 'font-size:16px;font-family:Arial'>Store number 1 for `StoreType` C</li> 
<li style = 'font-size:16px;font-family:Arial'>Store number 15 for `StoreType` D</li>

<p style = 'font-size:16px;font-family:Arial'>It also makes sense to down sample the data from days to weeks using the `resample` method to see the present trends more clearly.</p>

In [ ]:
train_df = sales_store.select(['Store','SalesDate','Sales'])

In [ ]:
sales_a = train_df[train_df.Store == 2].select(['SalesDate','Sales']).groupby('SalesDate').mean()
sales_b = train_df[train_df.Store == 85].select(['SalesDate','Sales']).groupby('SalesDate').sum()
# .sort_index(ascending = True) # solve the reverse order
sales_c = train_df[train_df.Store == 1].select(['SalesDate','Sales']).groupby('SalesDate').sum()
sales_d = train_df[train_df.Store == 15].select(['SalesDate','Sales']).groupby('SalesDate').sum()

In [ ]:
fig, axes = subplots(nrows=4, ncols=1)
 
plot = sales_a.plot(x=sales_a.SalesDate, y=sales_a.mean_Sales,
                          ax=axes[0], figure=fig, kind="line",xlabel='Sales Date', ylabel='Sales',
                          title="Sales for Store 2", color="blue",figsize=(1200, 1600))
 
plot = sales_b.plot(x=sales_b.SalesDate, y=sales_b.sum_Sales,
                          ax=axes[1],figure=fig, kind="line",xlabel='Sales Date', ylabel='Sales',
                          title="Sales for Store 85", color="blue")
 
plot = sales_c.plot(x=sales_c.SalesDate, y=sales_c.sum_Sales,
                          ax=axes[2],figure=fig, kind="line",xlabel='Sales Date', ylabel='Sales',
                          title="Sales for Store 1", color="blue")

plot = sales_d.plot(x=sales_d.SalesDate, y=sales_d.sum_Sales,
                          ax=axes[3],figure=fig, kind="line",xlabel='Sales Date', ylabel='Sales',
                          title="Sales for Store 15", color="blue")
 
# Display the plot.
plot.show()

<p style = 'font-size:16px;font-family:Arial'>Retail sales for all store types tend to peak for the Christmas season and then decline after the holidays.</p>

<p style = 'font-size:16px;font-family:Arial'>Next we check the Yearly trend for these Store Types thing to check the presence of a trend in series. Time series decomposition is the process of separating time series data into its core components. These components include a potential trend (overall rise or fall in the mean), seasonality (a recurring cycle), and the remaining random residual. Python’s statsmodels library has a method for time series decomposition called seasonal_decompose(). The model type parameter can either be additive or multiplicative, here we consider additive as If the seasonality’s amplitude is independent of the level then you should use the additive model. The "period" parameter is the number of observations in a seasonal cycle. For example, if you have daily observations, the period is 1. </p>

In [ ]:
# Sorting with 'date'
pd_sales_store = features_df
train_df = pd_sales_store.set_index('SalesDate')
# Sales datacheck
train_df['Sales'] = train_df['Sales'] * 1.0
# storewise sales data
sales_a = train_df[train_df.Store == 2]['Sales'].sort_index(ascending = True)
sales_b = train_df[train_df.Store == 85]['Sales'].sort_index(ascending = True) # solve the reverse order
sales_c = train_df[train_df.Store == 1]['Sales'].sort_index(ascending = True)
sales_d = train_df[train_df.Store == 15]['Sales'].sort_index(ascending = True)

In [ ]:
# Seasonal decompose
from statsmodels.tsa.seasonal import seasonal_decompose

f, (ax1, ax2, ax3, ax4) = plt.subplots(4, figsize = (15, 15))

# monthly
decomposition_a = seasonal_decompose(sales_a, model = 'additive', extrapolate_trend='freq', period=1)
decomposition_a.trend.plot(ax = ax1)

decomposition_b = seasonal_decompose(sales_b, model = 'additive', extrapolate_trend='freq', period=1)
decomposition_b.trend.plot( ax = ax2)

decomposition_c = seasonal_decompose(sales_c, model = 'additive', extrapolate_trend='freq', period=1)
decomposition_c.trend.plot( ax = ax3)

decomposition_d = seasonal_decompose(sales_d, model = 'additive', extrapolate_trend='freq', period=1)
decomposition_d.trend.plot( ax = ax4)

<p style = 'font-size:16px;font-family:Arial'>Overall sales follow similar Trend for all StoreTypes as seen above. There are spikes around the year end which indicate higher sales over the year end holiday season.</p>

<hr style="height:2px;border:none">
<p style = 'font-size:20px;font-family:Arial'><b>5. Creating the model and forecasting using Prophet in python (stoSalesForecastnew.py).</b></p>

<p style = 'font-size:16px;font-family:Arial'>Prophet is a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data. Prophet is robust to missing data and shifts in the trend, and typically handles outliers well.</p>
<p style = 'font-size:16px;font-family:Arial'>Prophet follows the sklearn model API. We create an instance of the Prophet class and then call its fit and predict methods.</p>


<p style = 'font-size:16px;font-family:Arial'>All the below steps which include the Prophet model are executed in the python in the file <a href="./stoSalesForecastnew.py">stoSalesForecastnew.py</a> file. We then use this py file in the Script command and get the forecasted values.</p>
<p style = 'font-size:16px;font-family:Arial'>The input to Prophet is always a dataframe with two columns: ds and y. The ds (datestamp) column should be of a format expected by Pandas, ideally YYYY-MM-DD for a date or YYYY-MM-DD HH:MM:SS for a timestamp. The y column must be numeric and represents the measurement we wish to forecast.</p>

<p style = 'font-size:16px;font-family:Arial'>The below code shows the creation of the Sales DataFrame and the holidays Dataframe which are used in the model creation and model fit.</p>

<p style = 'font-size:16px;font-family:Arial'><b>Create Sales data dataframe using data from Vantage</b></p>

```python 
# create Sales data 
sales = pd_sales_store.rename(columns = {'SalesDate': 'ds','Sales': 'y'})
```  

<p style = 'font-size:16px;font-family:Arial'><b>Create holidays dataframe</b></p>

```python
#create holidays dataframe
     

school_dates = df[df.SchoolHoliday == 1].loc[:, 'Date'].values

school = pd.DataFrame({'holiday': 'school_holiday',
                      'ds': pd.to_datetime(school_dates)})

holidays = school      
```

<p style = 'font-size:16px;font-family:Arial'>We fit the model by instantiating a new Prophet object. Any settings to the forecasting procedure are passed into the constructor. Then you call its fit method and pass in the historical dataframe(sales).</p>

<p style = 'font-size:16px;font-family:Arial'><b>Instantiate and fit model using Prophet</b></p>

```python

# Prophet implementation 
my_model = Prophet(interval_width = 0.95, 
                   holidays = holidays.head(50000))
my_model.fit(sales)                   
```

<p style = 'font-size:16px;font-family:Arial'>Predictions are then made on a dataframe with a column ds containing the dates for which a prediction is to be made. You can get a suitable dataframe that extends into the future a specified number of days using the helper method Prophet.make_future_dataframe. By default, it will also include the dates from the history, so we will see the model fit as well.<p>

<p style = 'font-size:16px;font-family:Arial'><b>Create future dates for forecasting</b></p>

```python
dt = min(sales['ds'].values)
date1 = datetime.datetime.strptime(dt, "%y/%m/%d").date()



#  # Subtract one month
start_date = date1 - relativedelta(months=1)

# Get man date and then get future dates for 1 month
dt1 = max(sales['ds'].values)
date2 = datetime.datetime.strptime(dt1, "%y/%m/%d").date()
# date2 = datetime.datetime.strptime(datetime_str, "%Y/%m/%dT%H:%M:%S.%f").date()
end_date = date2 + relativedelta(months=1)
# end_date= str(end_value)


# # date_range = pd.date_range(start_date, periods=num_days)
date_range = pd.date_range(str(start_date), str(end_date))

future_dates = pd.DataFrame({'ds': date_range})               
```

<p style = 'font-size:16px;font-family:Arial'>The predict method will assign each row in future a predicted value which it names yhat. If you pass in historical dates, it will provide an in-sample fit. The forecast object here is a new dataframe that includes the "yhat" column, which is the forecast values for sales, as well as columns for components and uncertainty intervals.<p>

<p style = 'font-size:16px;font-family:Arial'><b>Create dataframe with forecast values</b></p>

```python
# forecast
forecast = my_model.predict(future_dates.head(10000))               
```

<p style = 'font-size:16px;font-family:Arial'>The forecasted values will be sent back to Vantage using the Returns clause of the Script function as seen in the section below.<p>

<hr style="height:2px;border:none">
<p style = 'font-size:20px;font-family:Arial'><b>6. Using Script Command to get the forecasted values back to Vantage.</b></p>

<p style = 'font-size:16px;font-family:Arial'>The SCRIPT COMMAND requires the below elements 
<li style = 'font-size:16px;font-family:Arial'><b>ON clause: </b> The SCRIPT function can have only one ON clause (single input). The ON clause can be specified with no options or with: HASH BY, PARTITION BY, PARTITION BY ANY, an optional ORDER BY or LOCAL ORDER BY clause</li>
<li style = 'font-size:16px;font-family:Arial'><b>SCRIPT_COMMAND: </b>
The script to be executed. The SCRIPT_COMMAND is a required keyword.
    <li style = 'font-size:16px;font-family:Arial'><b>runtime_literal_command: </b>The parameters to SCRIPT_COMMAND can be an executable name followed by the script name and other inputs, or any valid LINUX command.</li>
<li style = 'font-size:16px;font-family:Arial'><b>RETURNS: </b>
    The names and types of the output columns returned by the script. <b>* </b>Specifies that all columns of the input table should be returned by the SCRIPT function.</li>
    
<p style = 'font-size:16px;font-family:Arial'>First we will create a dataset which can be passed to the Script function.</p>    

In [ ]:
qry='''CREATE SET TABLE Store_Sales_ID 
     (
      SlsID INTEGER,
      Store INTEGER,
      DayOfWeek INTEGER,
      SalesDate DATE FORMAT 'yyyy/mm/dd',
      Sales INTEGER,
      Customers INTEGER,
      SalesOpen INTEGER,
      Promo INTEGER,
      StateHoliday CHAR(1) CHARACTER SET LATIN NOT CASESPECIFIC,
      SchoolHoliday INTEGER)
      PRIMARY INDEX ( SlsID ); '''
qry1='''insert into DEMO_USER.Store_Sales_ID select 1,  Store ,
      DayOfWeek ,
      SalesDate ,
      Sales ,
      Customers ,
      SalesOpen ,
      Promo ,
      StateHoliday,
      SchoolHoliday  from DEMO_prophetSTO.Sales_Data where Store <= 5;'''
try:
    execute_sql(qry)
    execute_sql(qry1) 
except:
    db_drop_table('Store_Sales_ID')
    execute_sql(qry)
    execute_sql(qry1) 

In [ ]:
final_table_df2 = DataFrame('Store_Sales_ID')

<p style = 'font-size:16px;font-family:Arial'>First we set the Database for execution to the user database: demo_user in this case</p>

In [ ]:
database_name = 'demo_user'

<p style = 'font-size:16px;font-family:Arial'>Set the search path to the database where the file is installed</p>

In [ ]:
execute_sql(f"SET SESSION SEARCHUIFDBPATH = {database_name};")

<p style = 'font-size:16px;font-family:Arial'>Install the user script file on Vantage. In case of rerun if the file already exists we first remove it and then install again.</p>

In [ ]:
sto = Script(data = final_table_df2, script_name='stoSalesForecastnew.py',
             files_local_path= r".",
             script_command= f'tdpython3 ./{database_name}/stoSalesForecastnew.py', 
                         delimiter = "\t", nulls_first = False,
                         returns={"ds":TIMESTAMP(0), "yhat": FLOAT(), "yhat_lower": FLOAT(), "yhat_upper": FLOAT() , 
                                  "trend": FLOAT(), "weekly": FLOAT(), "yearly": FLOAT()} # }, #,  "Variable":VARCHAR(50)
                        #  data_order_column = "pkg_id", #charset='latin',
                        # data_partition_column="v_id",
                        #  is_local_order = False, sort_ascending=False
            )

try:
    sto.install_file(file_identifier='stoSalesForecastnew', file_name='stoSalesForecastnew.py', is_binary=False)
except:
    sto.remove_file(file_identifier='stoSalesForecastnew', force_remove=False)
    sto.install_file(file_identifier='stoSalesForecastnew', file_name='stoSalesForecastnew.py', is_binary=False)

<p style = 'font-size:16px;font-family:Arial'>Execute the script in SQL using SCRIPT command with the following SQL code:</p>
<p style = 'font-size:16px;font-family:Arial'>Since the entire process of model training , fitting and scoring takes place in the .py file when used in the script command the below query make take some time approximately 50-60 seconds.</p>

In [ ]:
STO_DF = sto.execute_script()
STO_DF

In [ ]:
STO_DF.shape

<p style = 'font-size:16px;font-family:Arial'>The output contains 1004 rows(1 for each date) and 7 columns.
<p style = 'font-size:16px;font-family:Arial'>The forecasting output contains information for:
</p>
<li style = 'font-size:16px;font-family:Arial'>The forecasted value (yhat)</li>
<li style = 'font-size:16px;font-family:Arial'>Range for the forecasted values (yhat_lower and yhat_upper)</li>
<li style = 'font-size:16px;font-family:Arial'>The overall trend for a given date (also incorporates seasonality)</li>
<li style = 'font-size:16px;font-family:Arial'>Additive terms to adjust the trend to get the forecasted value</li>

<p style = 'font-size:16px;font-family:Arial'>To plot the forecast Values we select only the required columns and convert the teradataml dataframe to pandas dataframe.</p>

In [ ]:
plot_output = STO_DF.to_pandas(all_rows=True).reset_index()
plot_output["ds"] = pd.to_datetime(plot_output['ds']).dt.date

In [ ]:
plot_output_forecast = plot_output[['ds','yhat','yhat_lower','yhat_upper']].sort_values('ds', ascending=True)
# .tail(300)
plot_output_forecast = plot_output_forecast.reset_index()
plot_output_forecast.head(5)

<p style = 'font-size:16px;font-family:Arial'>To plot the forecast Values and the confidence level we set the lower and upper bounds of the confidence interval to yhat_lower and yhat_upper.</p>

In [ ]:
import matplotlib.pyplot as plt



# Create the data for the line graph, including the x-values and the corresponding upper and lower bounds
x_values = plot_output_forecast['ds'].values
y_values = plot_output_forecast['yhat'].values
lower_bounds = plot_output_forecast['yhat_lower'].values
upper_bounds = plot_output_forecast['yhat_upper'].values

 
plt.figure(figsize=(12, 8))
# Plot the line graph
plt.plot(x_values, y_values, color='black', label='Forecast Values')
plt.fill_between(x_values, lower_bounds, upper_bounds, color='lightblue', alpha=0.3, label='Confidence Interval')

 

# Customize the plot

plt.xlabel('Date')
plt.ylabel('Forecast Values')
plt.title('Forecast Sales Values with Confidence Interval')
plt.legend()

 

# Show the plot
plt.show()

<p style = 'font-size:16px;font-family:Arial'>The above graph contains the Forecast values(black line) and the light blue area is the range of the lower(yhat_lower) and upper(yhat_upper) limits of the forecasted values.</p>

<p style = 'font-size:16px;font-family:Arial'>Below we will check the general trend , the Weekly trend and the yearly trend of the forecasted values</p>

In [ ]:
plot_trend = STO_DF[['ds','trend','weekly','yearly']].sort('ds', ascending=True)
plot_trend = plot_trend.assign(day_week = plot_trend.ds.cast(type_=TIMESTAMP).day_of_week())

In [ ]:
fig, axes = subplots(nrows=3, ncols=1)
 
plot = plot_trend.plot(x=plot_trend.ds, y=plot_trend.trend,
                          ax=axes[0], figure=fig, kind="line",xlabel='Sales Date', ylabel='Sales', 
                          xtick_format='YYYY-MM',title="Sales Trend", color="blue",figsize=(1200, 1200))
 
plot = plot_trend.plot(x=plot_trend.day_week, y=plot_trend.weekly,
                          ax=axes[1],figure=fig, kind="scatter",xlabel='Sales Date', ylabel='Sales',
                          title="Weekly Sales", color="blue")
 
plot = plot_trend.plot(x=plot_trend.ds, y=plot_trend.yearly,
                          ax=axes[2],figure=fig, kind="line",xlabel='Sales Date', ylabel='Sales',
                          xtick_format='YYYY-MM',title="Yearly Sales", color="blue")

# Display the plot.
plot.show()

<li style = 'font-size:16px;font-family:Arial'>The first plot illustrates a gradual decline in sales over time. Although the decrease may not seem significant on the graph, it is worth noting that the sales range from a minimum of 4723 to a maximum of 4729.</li>
<li style = 'font-size:16px;font-family:Arial'>The second plot highlights the volume of sales on a particular Day of the week.</li>
<li style = 'font-size:16px;font-family:Arial'>The third plot highlights distinct peaks in sales activity during the winter holidays and summer holidays. These periods are the busiest seasons, indicating an increase in sales volume during those times.</li>

<hr style="height:2px;border:none">
<p style = 'font-size:20px;font-family:Arial'><b>7. Conclusion:</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have trained and validated the Prophet model using the python script and used the Script Table Operator(STO) using data from Vantage. We get the forecasted data in Vantage using the python script.</p>

<hr style="height:2px;border:none">
<p style = 'font-size:20px;font-family:Arial'><b>8. Cleanup</b></p>
<p style = 'font-size:18px;font-family:Arial'><b>Work Tables</b></p>

In [ ]:
db_drop_table(table_name='Store_Sales_ID')

<p style = 'font-size:18px;font-family:Arial'><b>Databases and Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_ProphetSTO');" 
#Takes 20 seconds

In [ ]:
remove_context()

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2023, 2024. All Rights Reserved
        </div>
    </div>
</footer>